In [1]:
from flask import Flask, render_template
from flask_cors import CORS
import json 
from coin_gecko import get_coingecko_prices
from binance import get_binance_prices
from coin_layer import get_coinlayer_prices

In [7]:
with open("dataa.json","r") as f:
    data2 = json.load(f)

def model_preds():
    print("hey model_predd",data2)
    return data2

def format_cgd(cgd):
    dataf = {}
    for coin in cgd:
        if(coin["Symbol"]=="btc"):
             dataf["BTC"] = coin["Price"]
        if(coin["Symbol"]=="ltc"):
             dataf["LTC"] = coin["Price"]
        if(coin["Symbol"]=="ada"):
             dataf["ADA"] = coin["Price"]
        if(coin["Symbol"]=="eth"):
             dataf["ETH"] = coin["Price"]
        if(coin["Symbol"]=="usdt"):
             dataf["USDT"] = coin["Price"]
    
    return dataf
    
def format_bd(bd):
        return {"BTC":float(bd["BTCUSD"]),"LTC":float(bd["LTCUSD"]),"ADA":float(bd["ADAUSD"]),"ETH":float(bd["ETHUSD"]),"USDT":float(bd["USDTUSD"]),}

def opportunity(buy,price,sell,cost,coin):
    return{"coin":coin,
    "sellerMarket":buy,
    "buyPrice":price,
    "buyerMarket":sell,
    "sellPrice":cost }
    

def compare(m1,m2,coin):
    m1price = m1["data"][coin]
    m2price = m2["data"][coin]
    if(m1price != m2price): 
        if(m1price< m2price): 
            return opportunity(m1["Name"],m1price,m2["Name"],m2price,coin)
        else:
            return opportunity(m2["Name"],m2price,m1["Name"],m1price,coin)
    else: 
        return False
    
def generate_arbitrage_opportunities():
    opportunities = []

    with open("cgarb.json","r") as f:
        cgd = json.load(f)
    with open("barb.json","r") as f:
        bd = json.load(f)
    with open("clarb.json","r") as f:
        clfd = json.load(f)["rates"]

    # cgd = get_coingecko_prices()
    # print("cgd",cgd)
    # bd = get_binance_prices()
    # print("bd",bd)
    # clfdo = get_coinlayer_prices()
    # print("clfdo",clfdo)
    # clfd = clfdo["rates"]
    # print("clfd",clfd)

    cgfd = format_cgd(cgd)
    bfd = format_bd(bd)
    
    clf = {"Name":"CoinLayer","data":clfd}
    cgf = {"Name":"CoinGecko","data":cgfd}
    bf = {"Name":"Binance","data":bfd}

    for coin in clf["data"]:
        comps = [ compare(clf,cgf,coin),
             compare(clf,bf,coin),
             compare(cgf,bf,coin)]
            
        for comp in comps:
            if comp != 0:
                opportunities.append(comp)
    return opportunities


In [3]:
# generate_arbitrage_opportunities()

In [13]:
app = Flask(__name__)

CORS(app,resources={r"/*": {"origins": ["https://turkishfilms.github.io"]}})
data2 =generate_arbitrage_opportunities()




@app.route('/')
def home():
   
   return render_template('index.html',data=data2)

@app.route('/learn')
def a():
    return render_template('learn.html')

@app.route('/predictions')
def b():
    model_predictions = model_preds()
    return render_template('predictions.html',data=json.dumps(model_predictions))

@app.route('/preds')
def predictCoinTrend():
    return json.dumps(data2)

if(__name__ == "__main__"):
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2023 11:39:33] "GET / HTTP/1.1" 200 -
